## ganache

In [73]:
%%writefile ../src/Bank.sol
pragma solidity ^0.4.0;

contract MyBank{
    address owner;
    uint balance;
    
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }
    
    event Sent(address from, address to, uint amount );

    function forwardTo(address payable _receiver) public payable {
        balance -= msg.value;
        require(msg.sender == owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    function widthdrawAll() public {
        balance -= address(this).balance;
        require(msg.sender == owner);
        msg.sender.transfer(address(this).balance);
    }

}

Overwriting ../src/Bank.sol


In [74]:
!solc --abi --bin ../src/Bank.sol


======= ../src/Bank.sol:MyBank =======
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506000600181905550610367806100686000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad88146100765780633c459375146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610260565b005b6100fd600480360360208110156100e757600080fd5b8101908080359060200190929190505050610312565b005b60008060015447915091509091565b346001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146101775760

In [3]:
%%writefile ../src/BankDeploy.js

var Web3=require('web3');
var web3;


web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506000600181905550610367806100686000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad88146100765780633c459375146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610260565b005b6100fd600480360360208110156100e757600080fd5b8101908080359060200190929190505050610312565b005b60008060015447915091509091565b346001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461017757600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156101bd573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a150565b476001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146102c957600080fd5b3373ffffffffffffffffffffffffffffffffffffffff166108fc479081150290604051600060405180830381858888f1935050505015801561030f573d6000803e3d6000fd5b50565b80341461031e57600080fd5b806001600082825401925050819055505056fea2646970667358221220f3b523a844fcfde76ea1453f80b9304ef5f76671f2c62972468d51675c07103764736f6c63430006040033";
var _contract = new web3.eth.Contract(_abiArray);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/BankDeploy.js


In [4]:
!node ../src/BankDeploy.js

Deploying the contract from 0xaF011A85a937DE99642d50520C81192BE7139dD2
hash: 0xad4fa1bf13ae209437467ae25460dc7f7f71e7220a3c4c87c70970d7dd3e0af1
---> The contract deployed to: 0x8938CC3EF70911586026Dba9203d2991cE292986


In [91]:
%%writefile ../src/BankUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var abi =[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var addr = "0x4F847995B6472BAa800d66400c85fdb89c67868D";
var bank = new web3.eth.Contract(abi,addr);

bank.methods.getBalance().call().then(console.log);

// 입금 11111 wei, 222 wei를 하고 잔고 11333 wei 출력
bank.methods.deposit(11111)
    .send({from:"0x117732fBb3A00538eeA94c8C80d70e460490F1C3",gas:100000, value:11111})
    .then(bank.methods.deposit(222)
        .send({from:"0x117732fBb3A00538eeA94c8C80d70e460490F1C3",gas:100000, value:222})
        .then(bank.methods.getBalance().call()))




Overwriting ../src/BankUse.js


In [92]:
!node ../src/BankUse.js

Result { '0': '88999', '1': '90664' }


In [59]:
%%writefile ../src/BankUse2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var abi =[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var addr = "0x4F847995B6472BAa800d66400c85fdb89c67868D";
var bank = new web3.eth.Contract(abi,addr);

bank.methods.getBalance().call().then(console.log);

//자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙 잔고 11000 wei 출력,
//bank.methods.forwardTo("0x07B033B74C32804F8c2cc1879501e3C806cE2652").send({from:"0x117732fBb3A00538eeA94c8C80d70e460490F1C3",gas:100000, value:333});
bank.methods.getBalance().call();

//2번째 계정 증가분(+333wei) 출력
web3.eth.getBalance("0x07B033B74C32804F8c2cc1879501e3C806cE2652", (err,res) => {
    console.log("balance: " + res); 
});

Overwriting ../src/BankUse2.js


In [60]:
!node ../src/BankUse2.js

Result { '0': '21001', '1': '22666' }
balance: 100000000000000001998


In [104]:
%%writefile ../src/BankUse3.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var abi =[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var addr = "0x4F847995B6472BAa800d66400c85fdb89c67868D";
var bank = new web3.eth.Contract(abi,addr);

//전액 인출하고 11000 wei
bank.methods.widthdrawAll().send({from:"0x117732fBb3A00538eeA94c8C80d70e460490F1C3",gas:100000});
bank.methods.getBalance().call()

// 자신의 잔고 증가분 출력 (+11000 wei)
web3.eth.getBalance("0x117732fBb3A00538eeA94c8C80d70e460490F1C3", (err,res) => {
    console.log("balance: " + res); 
});

Overwriting ../src/BankUse3.js


In [105]:
!node ../src/BankUse3.js

balance: 99968576576999975669


## geth

In [125]:
!echo | set /p="var _compiled=" > ..\src\BankGeth.js
!solc --optimize --combined-json abi,bin,interface ../src/Bank.sol >> ../src/BankGeth.js

In [126]:
!type ..\src\BankGeth.js

var _compiled={"contracts":{"../src/Bank.sol:MyBank":{"abi":"[{\"inputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"address\",\"name\":\"from\",\"type\":\"address\"},{\"indexed\":false,\"internalType\":\"address\",\"name\":\"to\",\"type\":\"address\"},{\"indexed\":false,\"internalType\":\"uint256\",\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"Sent\",\"type\":\"event\"},{\"inputs\":[{\"internalType\":\"uint256\",\"name\":\"amount\",\"type\":\"uint256\"}],\"name\":\"deposit\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"address payable\",\"name\":\"_receiver\",\"type\":\"address\"}],\"name\":\"forwardTo\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getBalance\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"},{\"internalType\":\"uint256\",\"name\":\"

In [25]:
%%writefile ../src/BankDeployGeth.js
var primary = eth.accounts[0];

loadScript('../src/BankGeth.js')
contractName=Object.keys(_compiled.contracts)

_abi=JSON.parse(_compiled.contracts[contractName[0]].abi)
_code=_compiled.contracts[contractName[0]].bin
_class=eth.contract(_abi);

console.log('bin code: ', _code)
//this async way does not work from the Jupyter Notebook
_object=_class.new({from:primary,data:'0x'+_code,gas:1000000}, function(err, contract) {
  if (!err && contract.address)
    console.log("contractAddress: ", contract.address);
    console.log("transactionHash: ", contract.transactionHash);
});

Overwriting ../src/BankDeployGeth.js


In [26]:
!geth --exec "loadScript('../src/BankDeployGeth.js')" attach http://localhost:8345

bin code:  608060405234801561001057600080fd5b50600080546001600160a01b03191633178155600155610210806100356000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad88146100725780633c4593751461009a578063b6b55f25146100af575b600080fd5b34801561005057600080fd5b506100596100cc565b6040805192835260208301919091528051918290030190f35b6100986004803603602081101561008857600080fd5b50356001600160a01b03166100d4565b005b3480156100a657600080fd5b50610098610174565b610098600480360360208110156100c557600080fd5b50356101c3565b600154479091565b6001805434900390556000546001600160a01b031633146100f457600080fd5b6040516001600160a01b038216903480156108fc02916000818181858888f19350505050158015610129573d6000803e3d6000fd5b50604080513381526001600160a01b0383166020820152348183015290517f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd33459181900360600190a150565b6001805447900390556000546001600160a01b0316331461019457600080fd5b60405133904780156108fc02916000818181858888f193505050501580156101c0573d600

In [64]:
!geth --exec "eth.getTransactionReceipt('0x2ea81d76e6b757828a9e8cbab09570a41cb26931b3c2250d2347e7fce59e8c64')" attach http://localhost:8445

null


In [61]:
%%writefile ../src/BankUseGeth.js
loadScript('../src/BankGeth.js')
contractName=Object.keys(_compiled.contracts)

_abi=JSON.parse(_compiled.contracts[contractName[0]].abi)
var _contract=eth.contract(_abi);
var _address="0x521FfDda6fdcaBea180FdAe3a590581e87c8eF8C";
var _instance=eth.contract(_abi).at(_address);
console.log(_instance.getBalance.call());

Overwriting ../src/BankUseGeth.js


In [62]:
!geth --exec "loadScript('../src/BankUseGeth.js')" attach http://localhost:8345

GoError: Error while compiling or running script: Error: Reverting to invalid state checkpoint failed at web3.js:6347:37(47)
	at native
	at <eval>:1:12(1)



In [32]:
%%writefile ../src/BankDeployGeth.js

var Web3=require('web3');
var web3;


web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8447"));

var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506000600181905550610367806100686000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad88146100765780633c459375146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610260565b005b6100fd600480360360208110156100e757600080fd5b8101908080359060200190929190505050610312565b005b60008060015447915091509091565b346001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461017757600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156101bd573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a150565b476001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146102c957600080fd5b3373ffffffffffffffffffffffffffffffffffffffff166108fc479081150290604051600060405180830381858888f1935050505015801561030f573d6000803e3d6000fd5b50565b80341461031e57600080fd5b806001600082825401925050819055505056fea2646970667358221220f3b523a844fcfde76ea1453f80b9304ef5f76671f2c62972468d51675c07103764736f6c63430006040033";
var _contract = new web3.eth.Contract(_abiArray);

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    
    new web3.eth.Contract(_abiArray).deploy({data: '0x' + _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });
    
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: '0x' + _bin})
        .send({from: accounts[0], gas: 1000000, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting ../src/BankDeployGeth.js


```personal.unlockAccount(eth.accounts[0], "1111")```
contract could not code 어쩌구 에러는 마이닝으로 돈 계속 벌면 해결됨 좀 기다려야됨  

그리고 마이닝 ```miner.start(1);admin.sleepBlocks(1);miner.stop()```

In [33]:
!node ../src/BankDeployGeth.js

Deploying the contract from 0x5C46B33Dc12cF6ebbb834310EF235730f6C8449e
>> gas: 300800
hash: 0xe61d3357273223652da8fb8c7597f5b59cfc1c48ac67e4f0532cdb5174eafade
---> The contract deployed to: 0x1D800f26915e07c644363fCF959502d205AC8857


In [22]:
%%writefile ../src/BankUseGeth.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8447"));

var bank = new web3.eth.contract([{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}],"0x81eD10D4F4E353AE9cC8C59af1B34bB0bBa4eDa6");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log(accounts);
    const balance0Before = await web3.eth.getBalance(accounts[0]);
    console.log("Balance0 before: " + balance0Before);
    const balance1Before = await web3.eth.getBalance(accounts[1]);
    console.log("Balance0 before: " + balance1Before);
    
    bank.methods.getBalance().call().then(console.log)
}
doIt();

Overwriting ../src/BankUseGeth.js


In [23]:
!node ../src/BankUseGeth.js

[
  '0x31Dc76679415640194B68dfbDb301c7a4565Cb79',
  '0x6D81E0795e1b506f01886ba9a79Fe609DBAF203F',
  '0x97AC3bd7450282c18A32A8128FE1E1D8713F90AC',
  '0xff6EF3da460E4543558dC22a184e5beC5F3963D7',
  '0xCAef350f0046CfA92E780C10EEE58e5Db122d9c8',
  '0x8de4dF494dD53c7a70d0c7dd9B22ab9a273608a8',
  '0xbC8C97e0b6EeD23B7532Bf3a71E600fbD86D1Bed',
  '0x739D0a4Afab862Cb3D172e56d6B576c0b616D2cE',
  '0x93Ba32F2370062a8f1E6e66B3d8095a9C3601aF0',
  '0x1B99c28579581ab93e2D2Cc2590F52C0801d12D5'
]
Balance0 before: 99999735900000000000
Balance0 before: 100000000000000000000
Result { '0': '0', '1': '0' }


In [14]:
%%writefile ../src/BankUseGeth2.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8447"));

var abi =[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var addr = "0x1D800f26915e07c644363fCF959502d205AC8857";
var bank = new web3.eth.Contract(abi,addr);

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log(accounts);
    const balance0Before = await web3.eth.getBalance(accounts[0]);
    console.log("Balance0 before: " + balance0Before);
    const balance1Before = await web3.eth.getBalance(accounts[1]);
    console.log("Balance0 before: " + balance1Before);
    
    bank.methods.getBalance().call().then(console.log);
}
doIt();

Overwriting ../src/BankUseGeth2.js


In [15]:
!node ../src/BankUseGeth2.js

[
  '0x5C46B33Dc12cF6ebbb834310EF235730f6C8449e',
  '0x707C2942c4269BcaF5377B926D9FFc02610FD62d'
]
Balance0 before: 645000000000000000000
Balance0 before: 0


(node:10136) UnhandledPromiseRejectionWarning: Error: Returned error: invalid opcode: SHR
    at Object.ErrorResponse (C:\Users\yoonho\Code\201910812\node_modules\web3-core-helpers\src\errors.js:29:16)
    at C:\Users\yoonho\Code\201910812\node_modules\web3-core-requestmanager\src\index.js:140:36
    at XMLHttpRequest.request.onreadystatechange (C:\Users\yoonho\Code\201910812\node_modules\web3-providers-http\src\index.js:110:13)
    at XMLHttpRequestEventTarget.dispatchEvent (C:\Users\yoonho\Code\201910812\node_modules\xhr2-cookies\dist\xml-http-request-event-target.js:34:22)
    at XMLHttpRequest._setReadyState (C:\Users\yoonho\Code\201910812\node_modules\xhr2-cookies\dist\xml-http-request.js:208:14)
    at XMLHttpRequest._onHttpResponseEnd (C:\Users\yoonho\Code\201910812\node_modules\xhr2-cookies\dist\xml-http-request.js:318:14)
    at IncomingMessage.<anonymous> (C:\Users\yoonho\Code\201910812\node_modules\xhr2-cookies\dist\xml-http-request.js:289:61)
    at IncomingMessage.emit (ev